In [ ]:
from PIL import Image
import os
from random import randrange
import math
import threading
import time

In [ ]:
sliding_stride=1
kernel_size=3
layers=15
kernel=[[[0]*3 for i in range(kernel_size)] for j in range(kernel_size)]
for i in range(kernel_size):
   for j in range(kernel_size):
     for k in range(3):
       kernel[i][j][k]=randrange(255)
     kernel[i][j]=list(kernel[i][j])

In [ ]:
def convolution_loop(pixels,kernel,color,row,col):
   val=0
   for i in range(0,kernel_size,sliding_stride):
     for j in range(0,kernel_size,sliding_stride):
       val+=(kernel[i][j][color]*pixels[i+row][j+col][color])
   return val

In [ ]:
pixel_tiling=[[[0]*3 for i in range(1000)] for j in range(1000)]

In [ ]:
def convolution_loop_tiling(start_row,start_col,pixels,height,width,tile_size):
    #print(start_row,start_col,"start")
    
    pixel_new=[]
    for i in range(start_row,start_row+tile_size-kernel_size,sliding_stride):
        arr=[]
        for j in range(start_col,start_col+tile_size-kernel_size,sliding_stride):
            lis=[]
            for color in range(3):
              val=0
              for k in range(0,kernel_size,sliding_stride):
                #for l in range(0,kernel_size,sliding_stride):
                #  val+=(kernel[k][l][color]*pixels[i+k][j+l][color])
                val+=(kernel[k][0][color]*pixels[i+k][0+l][color])
                val+=(kernel[k][1][color]*pixels[i+k][1+l][color])
                val+=(kernel[k][2][color]*pixels[i+k][2+l][color])
              lis.append(val/256)
            lis=list(lis)
            arr.append(lis)
        pixel_new.append(arr)
    #return pixel_new  
    for i in range(len(pixel_new)):
        for j in range(len(pixel_new[0])):
            for k in range(3):
                pixel_tiling[start_row+i][start_col+j][k]=pixel_new[i][j][k]
    #pixel_tiling.append(pixel_new)
    #print(start_row,start_col,"end")

In [ ]:
c=0
start=time.time()
for file in os.listdir("/home/xilinx/jupyter_notebooks/train"):
  path=os.path.join("/home/xilinx/jupyter_notebooks/train",file)
  #print(path)
  im = Image.open(path)
  pixels = list(im.getdata())
  width, height = im.size 
  pixels = [pixels[i * width:(i + 1) * width] for i in range(height)]
  print(len(pixels),len(pixels[0]))
  for l in range(layers):
      pixel_tiling=[[[0]*3 for i in range(1000)] for j in range(1000)]
      tile_size=9
      tile_index=[]
      for i in range(0,height,tile_size):
        for j in range(0,width,tile_size):
            tile_index.append([i,j])
      tile_thread=[]
      for i in tile_index:
        if(height-i[0]>=tile_size and width-i[1]>=tile_size):
            t = threading.Thread(target=convolution_loop_tiling, args=(i[0],i[1],pixels,height,width,tile_size))
            t.start()

      main_thread = threading.current_thread()
      for t in threading.enumerate():
        if t is main_thread:
          continue
          t.join()
      arr=[]
      for i in range(len(pixel_tiling)):
            num=[]
            for j in range(len(pixel_tiling[0])):
                lis=[]
                for k in range(3):
                    if(pixel_tiling[i][j][k]!=0):
                        lis.append(pixel_tiling[i][j][k])
                if(len(lis)>0):
                    num.append(lis)
            if(len(num)>0):
                arr.append(num)
      height=len(arr)
      width=0
      if(len(arr)>0):
        width=len(arr[0])
      pixels=arr
      #print(pixels)
  c+=1
  if(c==1):
    break
end=time.time()        
                
      
  
  

In [ ]:
print((end-start)*(1000))